<a href="https://colab.research.google.com/github/Anjanisinghthakur/Google-Colab/blob/main/Hierarchical_grouping(groupBy%2Crollup%2Ccube).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("groupBy,rollup,cube(Totals,Subtotals, Grandtotals)").getOrCreate()
data=[
    ["Europe", "France", 1000],
    ["Europe", "Italy", 2000],
    ["Europe", "Germany", 2500],
    ["Asia","India",4000],
    ["Asia","China",1000],
    ["Asia","Korea",2000],
    ["Africe","Kenya",1000]
]
schema=StructType([
    StructField("Continent",StringType(),True),
    StructField("Country",StringType(),True),
    StructField("Profit",IntegerType(),True),

])
df=spark.createDataFrame(data,schema=schema)

res1=df.groupBy(df.Continent,df.Country).agg(sum(df.Profit).alias("sum_profit"))
res1=res1.orderBy(res1.Continent,res1.sum_profit)

res2=df.rollup(df.Continent,df.Country).agg(sum(df.Profit).alias("sum_profit"))
res2=res2.orderBy(res2.Continent,res2.sum_profit)

res3=df.cube(df.Continent,df.Country).agg(sum(df.Profit).alias("sum_profit"))
res3=res3.orderBy(res3.Continent,res3.sum_profit)

res1.show()
res2.show()
res3.show()

+---------+-------+----------+
|Continent|Country|sum_profit|
+---------+-------+----------+
|   Africe|  Kenya|      1000|
|     Asia|  China|      1000|
|     Asia|  Korea|      2000|
|     Asia|  India|      4000|
|   Europe| France|      1000|
|   Europe|  Italy|      2000|
|   Europe|Germany|      2500|
+---------+-------+----------+

+---------+-------+----------+
|Continent|Country|sum_profit|
+---------+-------+----------+
|     NULL|   NULL|     13500|
|   Africe|  Kenya|      1000|
|   Africe|   NULL|      1000|
|     Asia|  China|      1000|
|     Asia|  Korea|      2000|
|     Asia|  India|      4000|
|     Asia|   NULL|      7000|
|   Europe| France|      1000|
|   Europe|  Italy|      2000|
|   Europe|Germany|      2500|
|   Europe|   NULL|      5500|
+---------+-------+----------+

+---------+-------+----------+
|Continent|Country|sum_profit|
+---------+-------+----------+
|     NULL| France|      1000|
|     NULL|  Kenya|      1000|
|     NULL|  China|      1000|
|     